# Data Split: Train, Validation, Test,  sets

---

## Fundamentals of ETL: data extraction, transformation and loading


Applied Mathematical Modeling in Banking

---

# Table of contents

---

# 1. What's Train, Validation, Test datasets


Before model fitting and some stages of features engeniering we shoudl split out dataset on 2 or 3 parts:

- [x] `Training` dataset: The sample of data used to fit the model.

The model sees and learns from this data.

- [x] `Validation` dataset: The sample of data used to provide an unbiased evaluation of a model fit on the training dataset while tuning model hyperparameters. The evaluation becomes more biased as skill on the validation dataset is incorporated into the model configuration.

The validation set is used to evaluate a given model, but this is for frequent evaluation. We, as machine learning engineers, use this data to fine-tune the model hyperparameters. Hence the model occasionally sees this data, but never does it “Learn” from this. We use the validation set results, and update higher level hyperparameters. So the validation set affects a model, but only indirectly. The validation set is also known as the Dev set or the Development set. This makes sense since this dataset helps during the “development” stage of the model.

- [x] `Testing` dataset: Test Dataset: The sample of data used to provide an unbiased evaluation of a final model fit on the training dataset.

The Test dataset provides the gold standard used to evaluate the model. It is only used once a model is completely trained(using the train and validation sets). The test set is generally what is used to evaluate competing models (For example on many Kaggle competitions, the validation set is released initially along with the training set and the actual test set is only released when the competition is about to close, and it is the result of the the model on the Test set that decides the winner). Many a times the validation set is used as the test set, but it is not good practice. The test set is generally well curated. It contains carefully sampled data that spans the various classes that the model would face, when used in the real world.

![](assets/images/03/train_test1.png)

You can also find papers with splitting only for `train`/`test`. In this case `test` means `validation`.

![](assets/images/03/train_test2.png)

---

# 2. Splitting data in R

Lets describe some conditions before start studiyng splitting data functions in R:

- [x] We will use same seed for all splittings to control results reproduction, for example, let it be 2021.
- [x] We will use dataset for client churn prediction `Telco Customer Churn`: https://www.kaggle.com/blastchar/telco-customer-churn

Short dataset description:

- [x] `customerID` - Customer ID
- [x] `gender` Whether the customer is a male or a female
- [x] `SeniorCitizen` - Whether the customer is a senior citizen or not (1, 0)
- [x] `Partner` - Whether the customer has a partner or not (Yes, No)
- [x] `Dependents` - Whether the customer has dependents or not (Yes, No)
- [x] `tenure` - Number of months the customer has stayed with the company
- [x] `PhoneService` - Whether the customer has a phone service or not (Yes, No)
- [x] `MultipleLines` - Whether the customer has multiple lines or not (Yes, No, No phone service)
- [x] `InternetService` - Customer’s internet service provider (DSL, Fiber optic, No)
- [x] `OnlineSecurity` - Whether the customer has online security or not (Yes, No, No internet service)
- [x] `OnlineBackup` - Whether the customer has online backup or not (Yes, No, No internet service)
- [x] `DeviceProtection` - Whether the customer has device protection or not (Yes, No, No internet service)
- [x] `TechSupport` - Whether the customer has tech support or not (Yes, No, No internet service)
- [x] `StreamingTV` - Whether the customer has streaming TV or not (Yes, No, No internet service)
- [x] `StreamingMovies` - Whether the customer has streaming movies or not (Yes, No, No internet service)
- [x] `Contract` - The contract term of the customer (Month-to-month, One year, Two year)
- [x] `PaperlessBilling` - Whether the customer has paperless billing or not (Yes, No)
- [x] `PaymentMethod` - The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
- [x] `MonthlyCharges` - The amount charged to the customer monthly
- [x] `TotalCharges` - The total amount charged to the customer
- [x] `Churn` - Whether the customer churned or not (Yes or No)

In [2]:
# read data
telecom_users <- read.csv("data/telecom_users.csv")
head(telecom_users)

,X,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.1,1734.65,No
2,4528,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,...,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No
3,6344,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
4,6739,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.9,238.50,No
5,432,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No
6,2215,4312-GVYNH,Female,0,Yes,No,70,No,No phone service,DSL,...,Yes,Yes,No,Yes,Two year,Yes,Bank transfer (automatic),49.85,3370.20,No


Lets check the proportion of column `Churn == Yes` and `Churn == No` in dataset with `CrossTable()` function from `gmodels` package.

In [3]:
#install.packages("gmodels")
library(gmodels)
CrossTable(telecom_users$Churn)


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  5986 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      4399 |      1587 | 
          |     0.735 |     0.265 | 
          |-----------|-----------|



 


You can also use `CrossTable()` to check cross proportions by other fields. Lets check crosstable for `TechSupport` and `Churn`:

In [1]:
CrossTable(telecom_users$Churn, telecom_users$TechSupport) # for example

ERROR: Error in CrossTable(telecom_users$Churn, telecom_users$TechSupport): could not find function "CrossTable"


You can see that most part of Churn 1222 of 1587

Next, we will check 6 possible ways to split data for train/test sets.

---

## 2.1. Split with `sample()`

In [5]:
set.seed(2021)

sample_size = round(nrow(telecom_users)*.70) # setting what is 70%
print(paste0("Size: ", sample_size))

index <- sample(nrow(telecom_users), size = sample_size)
 
train <- telecom_users[index, ] # index is numbers of selected rows from dataset
test <-telecom_users[-index, ] # -index select only rows not in index

[1] "Size: 4190"


In [6]:
index # check the indexes and remember 1 and 2

[1] 4999 2214 5550  442  743 5004 1012 1094 4288 1787 5734 4421 4591  103
  [15]  361 5452 2995  956 5266 1147 1212 4733 2435 3886  360 4634 4260 2149
  [29] 4438 4767 1776 3426 3652 3926 4741 3145 3366 5394  683 2875  754 5979
  [43] 3414 4806 2751 2127 4770 2045 4091 5746  400  881 1624 1553 4838 1381
  [57] 5086 3528 1491 3139 3714 2833 4090 4777 5241 5919 2559 4465 3037 2203
  [71] 5624 2443 4718 1487 5842 3657 3603 1552 1506  362 4606 3830 2348  790
  [85]  550 1144 3853 2849  430 2709 4979 2649 2143 3757 3977  535 4331 3030
  [99]  420  606 3374 1799 4381 1683 5745 2468 5833 3187 4721 3401 1674 3704
 [113] 1345 2171 5317 4283 3731  844 1594 2055 2368 2220 5367 4235 5699 1044
 [127] 5783 1537  655 4469  992 5335 5229 4417 5491  961 5133 1734 3678 5205
 [141] 2350 3968 2494 2701  624 5621 1238 5790  392 5011  583 4440 4738 3637
 [155] 3925 1237 1462 3197 2125 4470 1093 4144 1992 1227 1720 4893 1973  227
 [169] 5331 2233 5975 4055 4990 4886  782 5590 5228 2326 1811 3021 5100 3677
 [183] 5817 5918 2202 2343 1606 2628  167 5922 2726 1573 1065  339 4003 2246
 [197] 2227 2810 3352 1835 2712 5304 4374 5115 1215 1257 1994 4646 5271  928
 [211] 3393 2105 1560 1432  712 4268  904 1360 3011  240 2540 3028 3100 1074
 [225] 2591 4084 3935  722 3613 4066  427 3057 1551 1944 3674 1206 4600 1770
 [239] 1822  884 5531 5641 3022  713  909 2502 5825 3621 5264 3133 2935 3479
 [253] 5374 2854 3775 2994 4081 3452 3389 5218 2869 4816  609 1533 1602 3768
 [267] 1763 4588 2324 2806 1233 4273 2520 5096 1293 1058 1609 3687 1967  279
 [281] 3063 3831 2323 2354 5262  894  177 2440  453 3126 4075 1974 5066 4698
 [295] 5212 2006 5028   98 1463 4783 5729  175 4446 5498 1331 3456 5253 2918
 [309] 1153 4295  601 2330  396 5865   76  174 2631 1642 3636 2914 3595 3166
 [323] 5439 1579  471 5379 5933 1340 2065 1318 4390 5943 3912 1114 2068 2947
 [337] 1968 2776 3702 4929 2981 5119 2529 1597 1970 1963 2509   17 4125 1543
 [351] 2889 2772 2571 1834 4905 4057  843  226 1269  390 4762 5376 2003 2002
 [365] 4585 1470  340 3284  317 3550 3432 1143 4123 3987 2817 5665 3920 3405
 [379] 5202 2921 4974 1982 3417 2676 3279 1046 1947 1988 5148 3196 3277  127
 [393] 3209 4252 4358 2164 5812  778 2664 3242  262 4982 3988 4214 2389 4498
 [407] 2111  855 1636 4833  944 5620   53 4581 3875 1535 4447  945 5419 5102
 [421] 3135 1855 2523 4532 5535  342 4860 1395 3317 4782   89 5853 3721  728
 [435] 4137 3526 4599 2903   12 4545  118 3908 4854 1388 5436 1324 1369 3703
 [449] 2166 3722 5192 4763 5774 4587 2014 1403   10 3959 4401 4707 1987 1503
 [463] 1347 5346  907 3570 3553 5293 5873 4822 5662 3123  670 3445 4044 3274
 [477] 2306 5632 5723  577 1613 5909 4747 5872 3073  350 4166 3693 4880 5390
 [491] 1567 4077  472 1765 4230 1846 5654 1201 1600 5905 4810 1539 2513 5272
 [505] 5269  185 4927 4203 1959 3962    8  230   77 3690 2647 4609 4403 5132
 [519]  451 3339 5977 1380 2253 2680  660 1359 3622  113 5754 1007 2655  908
 [533]  432 3089 5015 5232 3625  445 2361   47 1309 2103 5366  673 3131 5782
 [547] 3345 4024 4333 3305 3254 1710 5154 5935 5116 2699 5962 3578 5572 1934
 [561] 4958  129  531 4356 2448 3312 1723 5005 5818 2821 3010 3397 5234 2679
 [575] 5384 5032 3992 5118  765 3224 3536 4262 2075 1326  484 2094  468 5395
 [589] 2235 5560 3015 1766 5037 2454 5112 1205 5149 3162 3594 3730 5954 1467
 [603] 1952 3159 4119 5360  714  367 2497 1650 5256 1793 5831 4695 5431 3697
 [617]  115 5034 1327 3516 4005  565 1528 1865 5891 2195 2060  497 3746 4764
 [631] 4740 2909  559 5631 3341 3331 3986 3856 1438 1072 2946 3909 3665 4520
 [645] 3759 2118 4624 3177  691 2480 4459 5886 5137 1969 5794 4201 3266 1706
 [659] 2879 1248 3036 2782 2291 2405 1588 4797 2101 2084 1040 5174 5845 5957
 [673] 5074 3700 2900 2518  199 3736 1409 3428 1024 2862 1229 2616 2534 3671
 [687] 3435 3402 2999 3215 4668 4395  195 2762 3883 1121 5764 2877   84 3975
 [701] 3781  899 5290 5022 2116 1815 1559  679 3588  996  347 5800 4325 5013
 [715] 2962 1750  596 2356 2610   55 3929 2728 4219 4661  903 1729 5528 5479

In [7]:
# check Churn == Yes/No proportion in train/test
CrossTable(train$Churn)


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  4190 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      3099 |      1091 | 
          |     0.740 |     0.260 | 
          |-----------|-----------|



 


In [43]:
# check Churn == Yes/No proportion in train/test
CrossTable(test$Churn)


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1796 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      1300 |       496 | 
          |     0.724 |     0.276 | 
          |-----------|-----------|



 


Its 0.260 for train and 0.276 for test. Diffrence is 1,6%, so, its close.

## 2.2. Split with `sample.int()`

In [8]:
set.seed(2021)
 
sample <- sample.int(n = nrow(telecom_users),
                     size = floor(.70*nrow(telecom_users)), # Selecting 70% of data
                     replace = F)
 
train <- telecom_users[sample, ]
test  <- telecom_users[-sample, ]

# check Churn == Yes/No proportion in train/test
CrossTable(train$Churn)
CrossTable(test$Churn)


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  4190 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      3099 |      1091 | 
          |     0.740 |     0.260 | 
          |-----------|-----------|



 

 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1796 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      1300 |       496 | 
          |     0.724 |     0.276 | 
          |-----------|-----------|



 


In [9]:
index # compare indexes with previous function. Is it the same?

[1] 4999 2214 5550  442  743 5004 1012 1094 4288 1787 5734 4421 4591  103
  [15]  361 5452 2995  956 5266 1147 1212 4733 2435 3886  360 4634 4260 2149
  [29] 4438 4767 1776 3426 3652 3926 4741 3145 3366 5394  683 2875  754 5979
  [43] 3414 4806 2751 2127 4770 2045 4091 5746  400  881 1624 1553 4838 1381
  [57] 5086 3528 1491 3139 3714 2833 4090 4777 5241 5919 2559 4465 3037 2203
  [71] 5624 2443 4718 1487 5842 3657 3603 1552 1506  362 4606 3830 2348  790
  [85]  550 1144 3853 2849  430 2709 4979 2649 2143 3757 3977  535 4331 3030
  [99]  420  606 3374 1799 4381 1683 5745 2468 5833 3187 4721 3401 1674 3704
 [113] 1345 2171 5317 4283 3731  844 1594 2055 2368 2220 5367 4235 5699 1044
 [127] 5783 1537  655 4469  992 5335 5229 4417 5491  961 5133 1734 3678 5205
 [141] 2350 3968 2494 2701  624 5621 1238 5790  392 5011  583 4440 4738 3637
 [155] 3925 1237 1462 3197 2125 4470 1093 4144 1992 1227 1720 4893 1973  227
 [169] 5331 2233 5975 4055 4990 4886  782 5590 5228 2326 1811 3021 5100 3677
 [183] 5817 5918 2202 2343 1606 2628  167 5922 2726 1573 1065  339 4003 2246
 [197] 2227 2810 3352 1835 2712 5304 4374 5115 1215 1257 1994 4646 5271  928
 [211] 3393 2105 1560 1432  712 4268  904 1360 3011  240 2540 3028 3100 1074
 [225] 2591 4084 3935  722 3613 4066  427 3057 1551 1944 3674 1206 4600 1770
 [239] 1822  884 5531 5641 3022  713  909 2502 5825 3621 5264 3133 2935 3479
 [253] 5374 2854 3775 2994 4081 3452 3389 5218 2869 4816  609 1533 1602 3768
 [267] 1763 4588 2324 2806 1233 4273 2520 5096 1293 1058 1609 3687 1967  279
 [281] 3063 3831 2323 2354 5262  894  177 2440  453 3126 4075 1974 5066 4698
 [295] 5212 2006 5028   98 1463 4783 5729  175 4446 5498 1331 3456 5253 2918
 [309] 1153 4295  601 2330  396 5865   76  174 2631 1642 3636 2914 3595 3166
 [323] 5439 1579  471 5379 5933 1340 2065 1318 4390 5943 3912 1114 2068 2947
 [337] 1968 2776 3702 4929 2981 5119 2529 1597 1970 1963 2509   17 4125 1543
 [351] 2889 2772 2571 1834 4905 4057  843  226 1269  390 4762 5376 2003 2002
 [365] 4585 1470  340 3284  317 3550 3432 1143 4123 3987 2817 5665 3920 3405
 [379] 5202 2921 4974 1982 3417 2676 3279 1046 1947 1988 5148 3196 3277  127
 [393] 3209 4252 4358 2164 5812  778 2664 3242  262 4982 3988 4214 2389 4498
 [407] 2111  855 1636 4833  944 5620   53 4581 3875 1535 4447  945 5419 5102
 [421] 3135 1855 2523 4532 5535  342 4860 1395 3317 4782   89 5853 3721  728
 [435] 4137 3526 4599 2903   12 4545  118 3908 4854 1388 5436 1324 1369 3703
 [449] 2166 3722 5192 4763 5774 4587 2014 1403   10 3959 4401 4707 1987 1503
 [463] 1347 5346  907 3570 3553 5293 5873 4822 5662 3123  670 3445 4044 3274
 [477] 2306 5632 5723  577 1613 5909 4747 5872 3073  350 4166 3693 4880 5390
 [491] 1567 4077  472 1765 4230 1846 5654 1201 1600 5905 4810 1539 2513 5272
 [505] 5269  185 4927 4203 1959 3962    8  230   77 3690 2647 4609 4403 5132
 [519]  451 3339 5977 1380 2253 2680  660 1359 3622  113 5754 1007 2655  908
 [533]  432 3089 5015 5232 3625  445 2361   47 1309 2103 5366  673 3131 5782
 [547] 3345 4024 4333 3305 3254 1710 5154 5935 5116 2699 5962 3578 5572 1934
 [561] 4958  129  531 4356 2448 3312 1723 5005 5818 2821 3010 3397 5234 2679
 [575] 5384 5032 3992 5118  765 3224 3536 4262 2075 1326  484 2094  468 5395
 [589] 2235 5560 3015 1766 5037 2454 5112 1205 5149 3162 3594 3730 5954 1467
 [603] 1952 3159 4119 5360  714  367 2497 1650 5256 1793 5831 4695 5431 3697
 [617]  115 5034 1327 3516 4005  565 1528 1865 5891 2195 2060  497 3746 4764
 [631] 4740 2909  559 5631 3341 3331 3986 3856 1438 1072 2946 3909 3665 4520
 [645] 3759 2118 4624 3177  691 2480 4459 5886 5137 1969 5794 4201 3266 1706
 [659] 2879 1248 3036 2782 2291 2405 1588 4797 2101 2084 1040 5174 5845 5957
 [673] 5074 3700 2900 2518  199 3736 1409 3428 1024 2862 1229 2616 2534 3671
 [687] 3435 3402 2999 3215 4668 4395  195 2762 3883 1121 5764 2877   84 3975
 [701] 3781  899 5290 5022 2116 1815 1559  679 3588  996  347 5800 4325 5013
 [715] 2962 1750  596 2356 2610   55 3929 2728 4219 4661  903 1729 5528 5479

---

## 2.3. Split with `sample_n` from `dplyr`

In [12]:
library(dplyr)
set.seed(2021)

# Using the above function to create 70 - 30 slipt into test and train
sample_size = round(nrow(telecom_users)*.70) # setting what is 70%
print(paste0("Size: ", sample_size))

train <- sample_n(telecom_users, sample_size)
sample_id <- as.numeric(rownames(train)) # rownames() returns character so as.numeric
test <- telecom_users[-sample_id,]

[1] "Size: 4190"


In [13]:
# check Churn == Yes/No proportion in train/test
CrossTable(train$Churn)
CrossTable(test$Churn)


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  4190 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      3099 |      1091 | 
          |     0.740 |     0.260 | 
          |-----------|-----------|



 

 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1796 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      1341 |       455 | 
          |     0.747 |     0.253 | 
          |-----------|-----------|



 


`sample_n` made other proportion of `Churn == Yes/No` and difference just 0.7%.

---

## 2.4. Split with `sample_frac` from `dplyr`

In [14]:
library(dplyr)
set.seed(2021)
# Using the above function to create 70 - 30 slipt into test and train
train <- sample_frac(telecom_users, 0.7)
sample_id <- as.numeric(rownames(train)) # rownames() returns character so as.numeric
test <- telecom_users[-sample_id,]

In [15]:
# check Churn == Yes/No proportion in train/test
CrossTable(train$Churn)
CrossTable(test$Churn)


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  4190 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      3099 |      1091 | 
          |     0.740 |     0.260 | 
          |-----------|-----------|



 

 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1796 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      1341 |       455 | 
          |     0.747 |     0.253 | 
          |-----------|-----------|



 


Looks like `sample_n` and `sample_frac` use the same sample splitting algorithm.

---

## 2.5. Split with `createDataPartition()` from `caret`

In [17]:
# install.packages("caret")
library(caret)
set.seed(2021)
 
index = createDataPartition(telecom_users$Churn, p = 0.70, list = FALSE)
train = telecom_users[index, ]
test = telecom_users[-index, ]

In [18]:
# check Churn == Yes/No proportion in train/test
CrossTable(train$Churn)
CrossTable(test$Churn)


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  4191 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      3080 |      1111 | 
          |     0.735 |     0.265 | 
          |-----------|-----------|



 

 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1795 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      1319 |       476 | 
          |     0.735 |     0.265 | 
          |-----------|-----------|



 


In [19]:
head(train)

,X,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.1,1734.65,No
5,432,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No
9,1480,8898-KASCD,Male,0,No,No,39,No,No phone service,DSL,...,Yes,Yes,No,No,One year,No,Mailed check,35.55,1309.15,No
10,5137,8016-NCFVO,Male,1,No,No,55,Yes,Yes,Fiber optic,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,116.5,6382.55,No
11,3169,4578-PHJYZ,Male,0,Yes,Yes,52,Yes,No,DSL,...,Yes,Yes,Yes,No,One year,Yes,Electronic check,68.75,3482.85,No
12,4653,2091-MJTFX,Female,0,Yes,Yes,30,No,No phone service,DSL,...,No,Yes,Yes,Yes,Month-to-month,No,Credit card (automatic),51.2,1561.50,Yes


Ckeck the proportion of target variable. Caret trying to make the same split for both train and test. This is one of the best split methods in R.

---

## 2.5. Split with `sample.split` from `caTools`

In [20]:
#install.packages("caTools")
library(caTools)
 
set.seed(2021)
sample = sample.split(telecom_users$Churn, SplitRatio = .75)
train = subset(telecom_users, sample == TRUE)
test  = subset(telecom_users, sample == FALSE)

In [21]:
# check Churn == Yes/No proportion in train/test
CrossTable(train$Churn)
CrossTable(test$Churn)


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  4489 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      3299 |      1190 | 
          |     0.735 |     0.265 | 
          |-----------|-----------|



 

 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1497 

 
          |        No |       Yes | 
          |-----------|-----------|
          |      1100 |       397 | 
          |     0.735 |     0.265 | 
          |-----------|-----------|



 


In [22]:
head(train)

,X,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.1,1734.65,No
4,6739,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.9,238.50,No
5,432,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No
6,2215,4312-GVYNH,Female,0,Yes,No,70,No,No phone service,DSL,...,Yes,Yes,No,Yes,Two year,Yes,Bank transfer (automatic),49.85,3370.20,No
7,5260,2495-KZNFB,Female,0,No,No,33,Yes,Yes,Fiber optic,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,90.65,2989.60,No
8,6001,4367-NHWMM,Female,0,No,No,1,No,No phone service,DSL,...,No,No,No,No,Month-to-month,Yes,Mailed check,24.9,24.90,No


Look like caret and caTools uses the same splitting algorithm too. 

---

## Для нашого курсу це не потрібно поки! Переходимо до наступної теми


# 3. Splitting for n-folds

In [29]:
# read data again
library(caret)
telecom_users <- read.csv("data/telecom_users.csv")
nrow(telecom_users)
head(telecom_users)

[1] 5986

,X,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.1,1734.65,No
2,4528,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,...,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No
3,6344,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
4,6739,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.9,238.50,No
5,432,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No
6,2215,4312-GVYNH,Female,0,Yes,No,70,No,No phone service,DSL,...,Yes,Yes,No,Yes,Two year,Yes,Bank transfer (automatic),49.85,3370.20,No


In [33]:
folds <- createFolds(telecom_users)
folds

$Fold01
[1]  1 10

$Fold02
[1]  3  4 15

$Fold03
[1]  2 17

$Fold04
[1] 8

$Fold05
[1] 16 22

$Fold06
[1]  9 18

$Fold07
[1] 19 20

$Fold08
[1]  5 12 14

$Fold09
[1] 11

$Fold10
[1]  6  7 13 21

In [34]:
library(caret)
library(mlbench)
data(Sonar)
 
folds <- createFolds(Sonar$Class)
str(folds)

ERROR: Error in library(mlbench): there is no package called 'mlbench'


## References

1. About Train, Validation and Test Sets in Machine Learning by Tarang Shah. Url: https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7